# Get all items by location, analyze, and update Holdings type

This notebook fetches all items in a specified location and then updates the Holdings record type field in the Folio Holdings record.

## Preparations

### Install modules
Most modules that we will use are already installed on Google Colab. The below we need to actively install.

In [ ]:
# We will use folioclient to communicate with FOLIO: https://pypi.org/project/folioclient/

!pip install folioclient

In [ ]:
# We will use xlsxwriter to create Excel files: https://pypi.org/project/XlsxWriter/ 

!pip install xlsxwriter

### Import modules
Modules that are installed need to be imported, so that we can access them in our code.

In [ ]:
import requests
import pandas
from pandas import json_normalize
from folioclient import FolioClient
from getpass import getpass
from google.colab import files

## Connect to FOLIO
We log into FOLIO by providing the okapi URL and the tenant ID, as well as the username and password of a FOLIO user. FOLIO will return a token which contains information about which user we are acting as, and what permissions that user has.

Note that tokens and passwords are sensitive data which should always be kept secret and secure.

### Provide tenant information and credentials

In [ ]:
okapi_url = "YOUR URL"
tenant_id = "YOUR Tenant ID"

# The getpass module lets us enter credentials at runtime
username = getpass('Enter username here: ')
password = getpass('Enter password here: ')


### Log into FOLIO
We will use the FolioClient module to make requests to FOLIO.

In [ ]:
# This will log us into FOLIO, and temporarily store important information like the token in the background
try:
  folio_client = FolioClient(okapi_url, tenant_id, username, password)
  print(f"Successfully logged into FOLIO as user {username}")
except Exception as e:
  print(e)

## Design the search query
This is the part where you specify what data you want to get from FOLIO.

### Endpoint, object and page size
Specify the endpoint you want to search, what objects you expect to get back, and an appropriate page size.

Appropriate page size varies by endpoint. For /inventory/items the maximum is around 100, but for /item-storage/items you can get tens of thousands of records at a time.

# Full list of UUIDs in [FOLIO Locations and Service Points](https://docs.google.com/spreadsheets/d/1EqfmwPf9vAzf9EDJ5Lld9GxB80Nv-1TT34tr1lEPuic/edit#gid=0)

Location IDs:
*   Clapp microforms - 1st floor (8651492d-9122-4b5e-b135-5e0d448f3726)
*   Clapp Stacks (6af5255d-b7f4-4f8f-81c8-080086ae1c6d)
*   music cd (permanentLocationId==e46ce021-3fc7-43c8-acfc-26dfcb2b2a3e)
*   music seminar room (permanentLocationId==d87be268-d156-4219-85be-c848f720f6a6)
*   Online resource (6a4c713b-a233-488a-b992-5f2f43e21a43)
*   Art Periodicals (5f82a1d7-8e6e-4626-b94b-aa9ac821037d)

Holdings Type Ids:
*   Boundwith (814a2c82-ddd1-4f79-a953-28bc0f3397e6)
*   Electronic (996f93e2-5b5e-4cf2-9168-33ced1f95eed)
*   Monograph (03c9c400-b9e3-4a07-ac0e-05ab470233ed)
*   Multi-part monograph (dc35d0ae-e877-488b-8e97-6e41444e6d0a)
*   Physical (0c422f92-0f4d-4d32-8cbe-390ebc33a3e5)
*   Serial (e6da6c98-6dd0-41bc-8b4b-cfd4bbd9c3ae)
*   Unmapped (4effa3a8-c991-4882-b1ee-2ea7c7487820)





In [ ]:
endpoint = "/holdings-storage/holdings"
record_type = "holdingsRecords"
page_size = "1000"
query = '?query=cql.AllRecords=1 AND permanentLocationId==5f82a1d7-8e6e-4626-b94b-aa9ac821037d AND holdingsTypeId==4effa3a8-c991-4882-b1ee-2ea7c7487820' 

## Get the item data from FOLIO
This is where you actually fetch the data from FOLIO. 

If your question is quite simple, and only requires you to interact with one API, you can do this with one query as in the example below.
* What are the titles and barcodes of all the items with this effective location?

An example of a more complex question, which requires queries to multiple APIs, could be
* What are the names of all users who have ever borrowed a book from x location?

### Fetch all items matching your query

In [ ]:
# Create an empty list. This is the basket where you will store retrieved items.
data = []

# Fetch all the results from FOLIO, in batches of whatever your page size is
for i, record in enumerate(folio_client.folio_get_all(
    path = endpoint, key = record_type, query = query, limit = int(page_size))):
  
  # Put the results in the basket 
  data.append(record)

  # Print out progress
  page_number = int(i / int(page_size))
  if (i > 0) and i % int(page_size) == 0:
      print(f"Fetched {i} records in {page_number} pages.")

print(f"Done! Fetched {i+1} records in {page_number + 1} pages.\n")

## Work with the data
There is a lot you can do here: create different tables based on various critera, analyse the content of the data, rename headers, change the content of the data.

### Read the item data into a dataframe and inspect it
A dataframe is a table-like structure similar to what you might be used working with in for example Excel.

In [ ]:
# Read the json data into a dataframe
df = pandas.json_normalize(data)

# Inspect the headers in the dataframe
df.info()
df.head()

UPDATE holdings type on holding record


In [ ]:
updated_holdings = []

for holdings in data:
   holdings["holdingsTypeId"] = "e6da6c98-6dd0-41bc-8b4b-cfd4bbd9c3ae"
   updated_holdings.append(holdings)
# Electronic - 996f93e2-5b5e-4cf2-9168-33ced1f95eed
# Physical - 0c422f92-0f4d-4d32-8cbe-390ebc33a3e5
# Serial - e6da6c98-6dd0-41bc-8b4b-cfd4bbd9c3ae

#print(*updated_holdings[:100], sep="\n") the brackets say how many you will update out of the total; remove to do all; 5000 too many, 3000ok
print(*updated_holdings[:3000], sep="\n")

In [ ]:
test_holdings = updated_holdings[:3000]
print(test_holdings)

In [ ]:
successfully_updated = 0


for i, updated_holding in enumerate(test_holdings):
  uuid = updated_holding["id"]
  body = updated_holding
  response = requests.put(f"{okapi_url}/holdings-storage/holdings/{uuid}", json=body, headers=folio_client.okapi_headers)

  if response.status_code == 204:
      successfully_updated += 1
  else:
      print(response.status_code, response.text)

 # Print out progress
 
  if (i > 0) and i % 500 == 0:
      print(f"Completed {i} records.")

print(f"Successfully updated: {successfully_updated}")